In [ ]:
# !pip install bs4

In [4]:
from bs4 import BeautifulSoup as BS
import requests
from time import sleep
import numpy as np
import pandas as pd
import re
import pandas as pd
import gzip
import shutil
from datasets import load_dataset
import os
import zipfile
from io import BytesIO
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed


/home/lielbin/.conda/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
prefix_path = parent_dir

## IMDB data analysis with episode ratings

In [3]:
def download_and_extract(url, output_filename):
    """
    Downloads a gzip file from a URL and extracts it.
    
    Args:
        url (str): The URL to download the file from.
        output_filename (str): The name of the extracted file.
    """
    print(f"Downloading {url}...")
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        compressed_file = output_filename + ".gz"
        with open(compressed_file, "wb") as f:
            f.write(response.content)
        print(f"Downloaded {compressed_file}")
        
        # Decompress the gzip file
        with gzip.open(compressed_file, "rb") as f_in:
            with open(output_filename, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"Extracted to {output_filename}")
    else:
        print(f"Failed to download {url}. Status code: {response.status_code}")

In [4]:
def load_tsv_to_dataframe(file_path):
    """
    Loads a TSV file into a pandas DataFrame.
    
    Args:
        file_path (str): The path to the TSV file.
    
    Returns:
        pandas.DataFrame: The loaded DataFrame.
    """
    print(f"Loading {file_path} into a DataFrame...")
    df = pd.read_csv(file_path, sep="\t", dtype=str, na_values="\\N")
    print(f"Loaded {file_path} with {len(df)} rows.")
    return df

In [5]:
# # URLs of IMDb datasets
# datasets = {
#     "title_episode": "https://datasets.imdbws.com/title.episode.tsv.gz",
#     "title_ratings": "https://datasets.imdbws.com/title.ratings.tsv.gz",
#     "title_basics": "https://datasets.imdbws.com/title.basics.tsv.gz",
#     "title_akas": "https://datasets.imdbws.com/title.akas.tsv.gz"
# }

# if __name__ == "__main__":
#     dataframes = {}
#     for name, url in datasets.items():
#         output_filename = f"./data/{name}.tsv"
#         download_and_extract(url, output_filename)

In [5]:
df_titles =load_tsv_to_dataframe(os.path.join(prefix_path, "data", "title_akas.tsv"))
df_episodes = load_tsv_to_dataframe(os.path.join(prefix_path, "data", "title_episode.tsv"))
df_basics = load_tsv_to_dataframe(os.path.join(prefix_path, "data", "title_basics.tsv"))
df_rating = load_tsv_to_dataframe(os.path.join(prefix_path, "data", "title_ratings.tsv"))

Loading /sise/eliorsu-group/lielbin/Courses/The-Art-of-Analyzing-Big-Data/data/title_akas.tsv into a DataFrame...
Loaded /sise/eliorsu-group/lielbin/Courses/The-Art-of-Analyzing-Big-Data/data/title_akas.tsv with 50627962 rows.
Loading /sise/eliorsu-group/lielbin/Courses/The-Art-of-Analyzing-Big-Data/data/title_episode.tsv into a DataFrame...
Loaded /sise/eliorsu-group/lielbin/Courses/The-Art-of-Analyzing-Big-Data/data/title_episode.tsv with 8679293 rows.
Loading /sise/eliorsu-group/lielbin/Courses/The-Art-of-Analyzing-Big-Data/data/title_basics.tsv into a DataFrame...
Loaded /sise/eliorsu-group/lielbin/Courses/The-Art-of-Analyzing-Big-Data/data/title_basics.tsv with 11297218 rows.
Loading /sise/eliorsu-group/lielbin/Courses/The-Art-of-Analyzing-Big-Data/data/title_ratings.tsv into a DataFrame...
Loaded /sise/eliorsu-group/lielbin/Courses/The-Art-of-Analyzing-Big-Data/data/title_ratings.tsv with 1509635 rows.


In [7]:
df_titles.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita,NaN,NaN,original,NaN,1
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita,US,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0


In [8]:
df_rating.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2108
1,tt0000002,5.6,283
2,tt0000003,6.5,2131
3,tt0000004,5.3,182
4,tt0000005,6.2,2860


In [9]:
df_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,NaN,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [10]:
df_episodes.head()

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0031458,tt32857063,NaN,NaN
1,tt0041951,tt0041038,1,9
2,tt0042816,tt0989125,1,17
3,tt0042889,tt0989125,NaN,NaN
4,tt0043426,tt0040051,3,42


In [11]:
# # Step 1: Rename columns in df_episodes
# df_combined = df_episodes.rename(columns={
#     'tconst': 'tconst_episode',
#     'parentTconst': 'tconst_season'
# }).copy()

# # Step 2: Add title_episode and runtimeMinutes from df_basics
# df_combined = df_combined.merge(
#     df_basics[['tconst', 'primaryTitle', 'runtimeMinutes']].rename(columns={
#         'tconst': 'tconst_episode',
#         'primaryTitle': 'title_episode',
#         'runtimeMinutes': 'runtimeMinutes'
#     }),
#     on='tconst_episode',
#     how='left'
# )

# # Step 3: Add title_series from df_basics
# df_combined = df_combined.merge(
#     df_basics[['tconst', 'primaryTitle']].rename(columns={
#         'tconst': 'tconst_season',
#         'primaryTitle': 'title_series'
#     }),
#     on='tconst_season',
#     how='left'
# )

# # Step 4: Add averageRating_season and numVotes_series from df_rating
# df_combined = df_combined.merge(
#     df_rating.rename(columns={
#         'tconst': 'tconst_season', 
#         'averageRating': 'averageRating_season', 
#         'numVotes': 'numVotes_series'
#     }),
#     on='tconst_season',
#     how='left'
# )

# # Step 5: Add averageRating_episode and numVotes_episode from df_rating
# df_combined = df_combined.merge(
#     df_rating.rename(columns={
#         'tconst': 'tconst_episode', 
#         'averageRating': 'averageRating_episode', 
#         'numVotes': 'numVotes_episode'
#     }),
#     on='tconst_episode',
#     how='left'
# )

# # Step 6: Add Year_episode from df_basics
# df_combined = df_combined.merge(
#     df_basics[['tconst', 'startYear']].rename(columns={
#         'tconst': 'tconst_episode',
#         'startYear': 'Year_episode'
#     }),
#     on='tconst_episode',
#     how='left'
# )

In [14]:
# df_combined.to_csv("imdb_episodes_with_season.csv", index=False)

In [6]:
# df_imdb = df_combined
df_imdb = pd.read_csv(os.path.join(prefix_path, "data", "imdb_episodes_with_season.csv"))


/tmp/ipykernel_3382644/2655301701.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_imdb = pd.read_csv(os.path.join(prefix_path, "data", "imdb_episodes_with_season.csv"))


In [7]:
df_imdb.head(100)

,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode
0,tt0031458,tt32857063,NaN,NaN,El huésped del sevillano,86.0,Teatro lírico español,NaN,NaN,6.9,15.0,1970.0
1,tt0041951,tt0041038,1.0,9.0,The Tenderfeet,30.0,The Lone Ranger,7.7,3012.0,7.6,98.0,1949.0
2,tt0042816,tt0989125,1.0,17.0,Othello,143.0,BBC Sunday-Night Theatre,7.0,188.0,7.6,12.0,1950.0
3,tt0042889,tt0989125,NaN,NaN,The Tragedy of King Richard II/II,145.0,BBC Sunday-Night Theatre,7.0,188.0,NaN,NaN,1950.0
4,tt0043426,tt0040051,3.0,42.0,Coriolanus,60.0,Studio One,7.5,344.0,NaN,NaN,1951.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,tt0055130,tt0190181,2.0,5.0,Man Detained,59.0,The Edgar Wallace Mystery Theatre,7.7,213.0,6.9,49.0,1961.0
96,tt0055131,tt0161126,4.0,27.0,The Man Out There,55.0,Armchair Theatre,7.5,262.0,8.0,27.0,1961.0
97,tt0055132,tt0190181,2.0,2.0,Man at the Carlton Tower,57.0,The Edgar Wallace Mystery Theatre,7.7,213.0,6.6,92.0,1961.0
98,tt0055203,tt0161126,4.0,47.0,Murder Club,60.0,Armchair Theatre,7.5,262.0,NaN,NaN,1961.0


In [14]:
df_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8679293 entries, 0 to 8679292
Data columns (total 12 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   tconst_episode         object 
 1   tconst_season          object 
 2   seasonNumber           float64
 3   episodeNumber          float64
 4   title_episode          object 
 5   runtimeMinutes         object 
 6   title_series           object 
 7   averageRating_season   float64
 8   numVotes_series        float64
 9   averageRating_episode  float64
 10  numVotes_episode       float64
 11  Year_episode           float64
dtypes: float64(7), object(5)
memory usage: 794.6+ MB


In [4]:
df_subtitles = pd.read_csv(os.path.join(prefix_path, "data", "processed_subtitles.csv"))
df_subtitles.head(500)

,tconst_season,id,versioname,season,episode
0,tt0060028,187892,Star.Trek-.The.Next.Generation.S03E01.Evolution,3.0,1.0
1,tt0060028,187893,Star.Trek-.The.Next.Generation.S03E02.The.Ensi...,3.0,2.0
2,tt0060028,187894,Star.Trek-.The.Next.Generation.S03E03.The.Surv...,3.0,3.0
3,tt0060028,187895,Star.Trek-.The.Next.Generation.S03E04.Who.Watc...,3.0,4.0
4,tt0060028,187896,Star.Trek-.The.Next.Generation.S03E05.The.Bonding,3.0,5.0
...,...,...,...,...,...
495,tt0092455,188002,star.trek.tng.s06e25.720p.bluray.x264-geckos,6.0,25.0
496,tt0092455,189699,Star.Trek.TNG.S06E26.720p.BluRay.x264-GECKOS,6.0,26.0
497,tt0092455,224682,Star.Trek_.The.Next.Generation.S01E02.WEBRip,1.0,2.0
498,tt0092455,224683,Star.Trek_.The.Next.Generation.S01E03.WEBRip,1.0,3.0


In [16]:
df_imdb_sub = df_imdb[df_imdb['tconst_season'].isin(df_subtitles['tconst_season'])]

In [17]:
df_imdb_sub.head()

,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode
216,tt0059753,tt0060028,1.0,0.0,The Cage,63.0,Star Trek,8.4,95360.0,7.6,7428.0,1966.0
277,tt0061027,tt0060028,1.0,3.0,Where No Man Has Gone Before,50.0,Star Trek,8.4,95360.0,7.7,6688.0,1966.0
539,tt0066931,tt1466074,1.0,3.0,Dead Weight,76.0,Columbo,8.3,43873.0,6.9,3538.0,1971.0
540,tt0066932,tt1466074,1.0,1.0,Murder by the Book,76.0,Columbo,8.3,43873.0,7.7,5505.0,1971.0
541,tt0066934,tt1466074,1.0,4.0,Suitable for Framing,76.0,Columbo,8.3,43873.0,7.7,3788.0,1971.0


In [18]:
df_imdb_sub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115481 entries, 216 to 8679026
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   tconst_episode         115481 non-null  object 
 1   tconst_season          115481 non-null  object 
 2   seasonNumber           114706 non-null  float64
 3   episodeNumber          114706 non-null  float64
 4   title_episode          115481 non-null  object 
 5   runtimeMinutes         90520 non-null   object 
 6   title_series           115481 non-null  object 
 7   averageRating_season   115421 non-null  float64
 8   numVotes_series        115421 non-null  float64
 9   averageRating_episode  107700 non-null  float64
 10  numVotes_episode       107700 non-null  float64
 11  Year_episode           114618 non-null  float64
dtypes: float64(7), object(5)
memory usage: 11.5+ MB


In [19]:
df_imdb_sub['tconst_season'].nunique()

4053

In [20]:
df_imdb_sub[df_imdb_sub['seasonNumber'].isnull() | df_imdb_sub['episodeNumber'].isnull()]

,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode
245380,tt0737504,tt0094574,NaN,NaN,Episode dated 25 September 2001,NaN,Unsolved Mysteries,8.3,6880.0,NaN,NaN,2001.0
245381,tt0737505,tt0094574,NaN,NaN,100 Solved Unsolved Mysteries,NaN,Unsolved Mysteries,8.3,6880.0,7.5,12.0,1996.0
245386,tt0737510,tt0094574,NaN,NaN,Angels,NaN,Unsolved Mysteries,8.3,6880.0,NaN,NaN,1998.0
245389,tt0737513,tt0094574,NaN,NaN,Bad Friday,NaN,Unsolved Mysteries,8.3,6880.0,NaN,NaN,2001.0
245393,tt0737517,tt0094574,NaN,NaN,Bomb Shelter Friends,NaN,Unsolved Mysteries,8.3,6880.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8549369,tt9586200,tt2359704,NaN,NaN,The Beginning of Golden Wind,24.0,JoJo's Bizarre Adventure,8.5,39138.0,8.3,651.0,2019.0
8659268,tt9865320,tt2874692,NaN,NaN,Heart on My Sleeve,NaN,When Calls the Heart,7.9,15512.0,8.1,105.0,2019.0
8660108,tt9867660,tt2359704,NaN,NaN,Determination,24,JoJo's Bizarre Adventure,8.5,39138.0,7.8,339.0,2019.0
8678494,tt9914702,tt0094574,NaN,NaN,Episode dated 5 February 1988,NaN,Unsolved Mysteries,8.3,6880.0,7.1,32.0,1988.0


In [21]:
df_imdb_sub.dropna(subset=['seasonNumber', 'episodeNumber'], inplace=True)

/tmp/ipykernel_3445768/3756378167.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imdb_sub.dropna(subset=['seasonNumber', 'episodeNumber'], inplace=True)


In [22]:
df_imdb_sub[df_imdb_sub['seasonNumber'].isnull() | df_imdb_sub['episodeNumber'].isnull()]

,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode


In [23]:
df_imdb_sub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114706 entries, 216 to 8679023
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   tconst_episode         114706 non-null  object 
 1   tconst_season          114706 non-null  object 
 2   seasonNumber           114706 non-null  float64
 3   episodeNumber          114706 non-null  float64
 4   title_episode          114706 non-null  object 
 5   runtimeMinutes         90361 non-null   object 
 6   title_series           114706 non-null  object 
 7   averageRating_season   114650 non-null  float64
 8   numVotes_series        114650 non-null  float64
 9   averageRating_episode  107547 non-null  float64
 10  numVotes_episode       107547 non-null  float64
 11  Year_episode           113914 non-null  float64
dtypes: float64(7), object(5)
memory usage: 11.4+ MB


In [28]:
# Ensure season and episode columns in df_subtitles are numeric and without decimals
df_subtitles['season'] = df_subtitles['season'].astype(float).astype('Int64')  # Removes .0
df_subtitles['episode'] = df_subtitles['episode'].astype(float).astype('Int64')

# Function to safely convert seasonNumber and episodeNumber to integers in df_imdb_sub
df_imdb_sub.loc[:, 'seasonNumber'] = pd.to_numeric(df_imdb_sub['seasonNumber'], errors='coerce').fillna(-1).astype('Int64')
df_imdb_sub.loc[:, 'episodeNumber'] = pd.to_numeric(df_imdb_sub['episodeNumber'], errors='coerce').fillna(-1).astype('Int64')

# Merge operation based on the conditions
df_imdb_sub = df_imdb_sub.merge(
    df_subtitles[['tconst_season', 'season', 'episode', 'id']],
    how='left',
    left_on=['tconst_season', 'seasonNumber', 'episodeNumber'],
    right_on=['tconst_season', 'season', 'episode']
)

# Rename the 'id' column from df_subtitles to 'sub_id' in df_imdb_sub
df_imdb_sub.rename(columns={'id': 'sub_id'}, inplace=True)

# Drop extra columns if needed (e.g., 'season', 'episode' from df_subtitles)
df_imdb_sub.drop(columns=['season', 'episode'], inplace=True)

In [30]:
df_imdb_sub = df_imdb_sub.loc[:, ~df_imdb_sub.columns.duplicated()]

In [34]:
df_imdb_sub = df_imdb_sub.drop_duplicates()

In [35]:
df_imdb_sub.head()

,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode,sub_id
0,tt0059753,tt0060028,1,0,The Cage,63.0,Star Trek,8.4,95360.0,7.6,7428.0,1966.0,188032.0
1,tt0061027,tt0060028,1,3,Where No Man Has Gone Before,50.0,Star Trek,8.4,95360.0,7.7,6688.0,1966.0,181335.0
17,tt0066931,tt1466074,1,3,Dead Weight,76.0,Columbo,8.3,43873.0,6.9,3538.0,1971.0,94598.0
18,tt0066932,tt1466074,1,1,Murder by the Book,76.0,Columbo,8.3,43873.0,7.7,5505.0,1971.0,NaN
19,tt0066934,tt1466074,1,4,Suitable for Framing,76.0,Columbo,8.3,43873.0,7.7,3788.0,1971.0,NaN


In [36]:
df_imdb_sub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114706 entries, 0 to 960379
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   tconst_episode         114706 non-null  object 
 1   tconst_season          114706 non-null  object 
 2   seasonNumber           114706 non-null  Int64  
 3   episodeNumber          114706 non-null  Int64  
 4   title_episode          114706 non-null  object 
 5   runtimeMinutes         90361 non-null   object 
 6   title_series           114706 non-null  object 
 7   averageRating_season   114650 non-null  float64
 8   numVotes_series        114650 non-null  float64
 9   averageRating_episode  107547 non-null  float64
 10  numVotes_episode       107547 non-null  float64
 11  Year_episode           113914 non-null  float64
 12  sub_id                 67119 non-null   float64
dtypes: Int64(2), float64(6), object(5)
memory usage: 12.5+ MB


In [37]:
df_imdb_sub.to_csv(os.path.join(prefix_path, "data", "imdb_subid.csv"), index=False)

In [38]:
df_imdb_sub['sub_id'].isnull().sum()

47587

In [39]:
df_imdb_sub = df_imdb_sub.dropna(subset=['sub_id'])

In [40]:
df_imdb_sub.to_csv(os.path.join(prefix_path, "data", "imdb_subid_notnull.csv"), index=False)

In [43]:
df_imdb_sub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67119 entries, 0 to 875904
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tconst_episode         67119 non-null  object 
 1   tconst_season          67119 non-null  object 
 2   seasonNumber           67119 non-null  Int64  
 3   episodeNumber          67119 non-null  Int64  
 4   title_episode          67119 non-null  object 
 5   runtimeMinutes         55401 non-null  object 
 6   title_series           67119 non-null  object 
 7   averageRating_season   67080 non-null  float64
 8   numVotes_series        67080 non-null  float64
 9   averageRating_episode  66047 non-null  float64
 10  numVotes_episode       66047 non-null  float64
 11  Year_episode           67054 non-null  float64
 12  sub_id                 67119 non-null  float64
dtypes: Int64(2), float64(6), object(5)
memory usage: 7.3+ MB


In [8]:
df_imdb_sub = pd.read_csv(os.path.join(prefix_path, "data", "imdb_subid_notnull.csv"))


In [9]:
df_imdb_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67119 entries, 0 to 67118
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tconst_episode         67119 non-null  object 
 1   tconst_season          67119 non-null  object 
 2   seasonNumber           67119 non-null  int64  
 3   episodeNumber          67119 non-null  int64  
 4   title_episode          67119 non-null  object 
 5   runtimeMinutes         55401 non-null  float64
 6   title_series           67119 non-null  object 
 7   averageRating_season   67080 non-null  float64
 8   numVotes_series        67080 non-null  float64
 9   averageRating_episode  66047 non-null  float64
 10  numVotes_episode       66047 non-null  float64
 11  Year_episode           67054 non-null  float64
 12  sub_id                 67119 non-null  float64
dtypes: float64(7), int64(2), object(4)
memory usage: 6.7+ MB


In [10]:
df_imdb_sub[df_imdb_sub['runtimeMinutes'].isnull()]

,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode,sub_id
403,tt0506500,tt0115086,1,6,Little Miss Magic,NaN,The Adventures of Sinbad,7.0,2252.0,7.4,40.0,1996.0,285900.0
404,tt0506502,tt0115086,1,1,Return of Sinbad: Part 1,NaN,The Adventures of Sinbad,7.0,2252.0,7.4,53.0,1996.0,285895.0
405,tt0506503,tt0115086,1,2,Return of Sinbad: Part 2,NaN,The Adventures of Sinbad,7.0,2252.0,7.0,46.0,1996.0,285896.0
406,tt0506505,tt0115086,1,4,Still Life,NaN,The Adventures of Sinbad,7.0,2252.0,7.1,44.0,1996.0,285898.0
407,tt0506513,tt0115086,1,5,The Ronin,NaN,The Adventures of Sinbad,7.0,2252.0,7.1,40.0,1996.0,285899.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66969,tt6721632,tt3110590,5,8,Caterham 7,NaN,Car S.O.S.,8.1,966.0,8.6,15.0,2017.0,311120.0
66970,tt6721642,tt3110590,5,9,Austin 'Tilly' Truck,NaN,Car S.O.S.,8.1,966.0,8.6,16.0,2017.0,311121.0
66971,tt6721648,tt3110590,5,10,Mazda RX7,NaN,Car S.O.S.,8.1,966.0,8.7,21.0,2017.0,311122.0
66972,tt6721728,tt4998212,3,1,Starting Over Again,NaN,The Ranch,7.5,43488.0,7.9,404.0,2018.0,203367.0


In [11]:
# Calculate the average runtimeMinutes for each tconst_season group
season_avg_runtime = df_imdb_sub.groupby('tconst_season')['runtimeMinutes'].mean()

#Define a function to fill null runtimeMinutes with the average for the season
def fill_runtime(row):
    if pd.isnull(row['runtimeMinutes']):
        return season_avg_runtime[row['tconst_season']]
    return row['runtimeMinutes']

# Apply the function to the dataframe
df_imdb_sub['runtimeMinutes'] = df_imdb_sub.apply(fill_runtime, axis=1)


In [12]:
df_imdb_sub[df_imdb_sub['runtimeMinutes'].isnull()]

,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode,sub_id
403,tt0506500,tt0115086,1,6,Little Miss Magic,NaN,The Adventures of Sinbad,7.0,2252.0,7.4,40.0,1996.0,285900.0
404,tt0506502,tt0115086,1,1,Return of Sinbad: Part 1,NaN,The Adventures of Sinbad,7.0,2252.0,7.4,53.0,1996.0,285895.0
405,tt0506503,tt0115086,1,2,Return of Sinbad: Part 2,NaN,The Adventures of Sinbad,7.0,2252.0,7.0,46.0,1996.0,285896.0
406,tt0506505,tt0115086,1,4,Still Life,NaN,The Adventures of Sinbad,7.0,2252.0,7.1,44.0,1996.0,285898.0
407,tt0506513,tt0115086,1,5,The Ronin,NaN,The Adventures of Sinbad,7.0,2252.0,7.1,40.0,1996.0,285899.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66882,tt6692646,tt6495756,1,13,The Face of the Killer,NaN,Ingobernable,7.0,3518.0,7.8,62.0,2017.0,281143.0
66883,tt6692648,tt6495756,1,14,Death Asks for Permission,NaN,Ingobernable,7.0,3518.0,7.8,59.0,2017.0,281144.0
66884,tt6692652,tt6495756,1,15,For the Sake of Justice,NaN,Ingobernable,7.0,3518.0,7.8,71.0,2017.0,281145.0
66951,tt6718794,tt4686698,2,1,Dead Reckoning,NaN,Frontier,7.1,24816.0,7.5,454.0,2017.0,279255.0


In [13]:
# Calculate the global median runtimeMinutes
global_median_runtime = df_imdb_sub['runtimeMinutes'].median()

# Fill remaining NULL values with the global median
df_imdb_sub['runtimeMinutes'] = df_imdb_sub['runtimeMinutes'].fillna(global_median_runtime)



In [14]:
df_imdb_sub[df_imdb_sub['Year_episode'].isnull()]

,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode,sub_id
11088,tt10006482,tt8027624,1,8,We Will Not Be Silenced,42.0,Tijuana,6.9,380.0,6.9,17.0,NaN,290983.0
11105,tt10009982,tt8027624,1,9,The Journalist's Path,42.0,Tijuana,6.9,380.0,6.9,17.0,NaN,290984.0
11106,tt10010014,tt8027624,1,10,The Journalist's Path,42.0,Tijuana,6.9,380.0,6.9,17.0,NaN,290985.0
11107,tt10010054,tt8027624,1,11,Change is the Only Constant,42.0,Tijuana,6.9,380.0,7.0,25.0,NaN,290986.0
14610,tt10876072,tt2181679,2,5,Silly Goose,42.0,Unlikely Animal Friends,7.3,123.0,NaN,NaN,NaN,310103.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59586,tt4839944,tt4760616,1,2,Echoes of the Past,45.0,Cape Town,6.1,396.0,NaN,NaN,NaN,39376.0
59587,tt4839946,tt4760616,1,3,Shallow End,45.0,Cape Town,6.1,396.0,NaN,NaN,NaN,39385.0
59588,tt4839948,tt4760616,1,5,Way to Paradise,46.0,Cape Town,6.1,396.0,NaN,NaN,NaN,39397.0
59589,tt4839950,tt4760616,1,4,Show-Time,45.0,Cape Town,6.1,396.0,NaN,NaN,NaN,39388.0


In [15]:
df_imdb_sub = df_imdb_sub[df_imdb_sub['Year_episode'].notnull()]

In [23]:
df_imdb_sub = df_imdb_sub.dropna(subset=['averageRating_episode'])

In [25]:
df_imdb_sub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66043 entries, 0 to 67118
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tconst_episode         66043 non-null  object 
 1   tconst_season          66043 non-null  object 
 2   seasonNumber           66043 non-null  int64  
 3   episodeNumber          66043 non-null  int64  
 4   title_episode          66043 non-null  object 
 5   runtimeMinutes         66043 non-null  float64
 6   title_series           66043 non-null  object 
 7   averageRating_season   66043 non-null  float64
 8   numVotes_series        66043 non-null  float64
 9   averageRating_episode  66043 non-null  float64
 10  numVotes_episode       66043 non-null  float64
 11  Year_episode           66043 non-null  float64
 12  sub_id                 66043 non-null  float64
dtypes: float64(7), int64(2), object(4)
memory usage: 7.1+ MB


In [27]:
print(df_imdb_sub.isnull().sum())

tconst_episode           0
tconst_season            0
seasonNumber             0
episodeNumber            0
title_episode            0
runtimeMinutes           0
title_series             0
averageRating_season     0
numVotes_series          0
averageRating_episode    0
numVotes_episode         0
Year_episode             0
sub_id                   0
dtype: int64


In [28]:
df_imdb_sub['genres'] = df_imdb_sub['tconst_episode'].map(
    df_basics.set_index('tconst')['genres']
)

/tmp/ipykernel_3382644/122469670.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imdb_sub['genres'] = df_imdb_sub['tconst_episode'].map(


In [31]:
df_imdb_sub = df_imdb_sub.dropna(subset=['genres'])

In [9]:
# Remove duplicates, keeping the first occurrence
df_imdb_sub = df_imdb_sub.drop_duplicates(subset=['sub_id'])

In [10]:
df_imdb_sub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66023 entries, 0 to 66034
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tconst_episode         66023 non-null  object 
 1   tconst_season          66023 non-null  object 
 2   seasonNumber           66023 non-null  int64  
 3   episodeNumber          66023 non-null  int64  
 4   title_episode          66023 non-null  object 
 5   runtimeMinutes         66023 non-null  float64
 6   title_series           66023 non-null  object 
 7   averageRating_season   66023 non-null  float64
 8   numVotes_series        66023 non-null  float64
 9   averageRating_episode  66023 non-null  float64
 10  numVotes_episode       66023 non-null  float64
 11  Year_episode           66023 non-null  float64
 12  sub_id                 66023 non-null  float64
 13  genres                 66023 non-null  object 
dtypes: float64(7), int64(2), object(5)
memory usage: 7.6+ MB


In [11]:
df_imdb_sub.to_csv(os.path.join(prefix_path, "data", "tv_imdb_episode_data.csv"), index=False)

In [41]:
# def get_and_process_subtitle(sub_id):
#     """
#     Fetches the subtitle file from the Wizdom API, processes it in memory,
#     and handles plain text, gzip, or zip files.
    
#     Args:
#         sub_id (int): The subtitle ID to fetch.
        
#     Returns:
#         str: The subtitle content in plain text format (SRT).
#     """
#     base_url = "https://wizdom.xyz/api/files/sub/"
#     url = f"{base_url}{sub_id}"
    
#     try:
#         # Fetch the file from the API
#         response = requests.get(url)
#         response.raise_for_status()

#         # Check for gzip-compressed content
#         if response.content[:2] == b'\x1f\x8b':  # Magic number for gzip
#             with gzip.GzipFile(fileobj=BytesIO(response.content)) as f:
#                 subtitle_content = f.read().decode('utf-8')
#                 return subtitle_content  # Return the decompressed SRT content
        
#         # Check for a zip archive
#         elif zipfile.is_zipfile(BytesIO(response.content)):
#             with zipfile.ZipFile(BytesIO(response.content)) as zip_file:
#                 for file_name in zip_file.namelist():
#                     with zip_file.open(file_name) as extracted_file:
#                         subtitle_content = extracted_file.read().decode('utf-8')
#                         return subtitle_content  # Return the extracted SRT content
        
#         # If it's plain text (assumed to be SRT)
#         else:
#             subtitle_content = response.content.decode('utf-8')
#             return subtitle_content  # Return plain SRT content

#     except requests.exceptions.RequestException as e:
#         print(f"An error occurred: {e}")
#         return None
#     except UnicodeDecodeError:
#         print("Failed to decode the content. It may not be text.")
#         return None


In [6]:
def get_and_process_subtitle(sub_id):
    """
    Fetches the subtitle file, processes it in memory,
    and handles plain text, gzip, or zip files with fallback decoding.
    
    Args:
        sub_id (int): The subtitle ID to fetch.
        
    Returns:
        str: The subtitle content in plain text format (SRT) or None if failed.
    """
    base_url = "https://wizdom.xyz/api/files/sub/"
    url = f"{base_url}{sub_id}"
    
    def decode_content(content):
        """
        Tries decoding content using UTF-8, then falls back to ISO-8859-8 or Windows-1255.
        """
        for encoding in ['utf-8', 'iso-8859-8', 'windows-1255']:
            try:
                return content.decode(encoding)
            except UnicodeDecodeError:
                continue
        return None  # Return None if decoding fails
    
    try:
        # Fetch the file from the API
        response = requests.get(url)
        response.raise_for_status()
        content = response.content

        # Check for gzip-compressed content
        if content[:2] == b'\x1f\x8b':  # Magic number for gzip
            with gzip.GzipFile(fileobj=BytesIO(content)) as f:
                decompressed_content = f.read()
                return decode_content(decompressed_content)

        # Check for zip archive
        elif zipfile.is_zipfile(BytesIO(content)):
            with zipfile.ZipFile(BytesIO(content)) as zip_file:
                for file_name in zip_file.namelist():
                    with zip_file.open(file_name) as extracted_file:
                        extracted_content = extracted_file.read()
                        return decode_content(extracted_content)

        # Try plain text decoding
        return decode_content(content)

    except requests.exceptions.RequestException as e:
        tqdm.write(f"Error while fetching subtitle for sub_id {sub_id}: {e}")
    except Exception as e:
        tqdm.write(f"Error processing subtitle for sub_id {sub_id}: {e}")
    
    return None

In [79]:
print(get_and_process_subtitle(188032.0)[:450])


1
00:00:26,908 --> 00:00:28,910
הכלוב

2
00:00:58,378 --> 00:01:00,381
.תבדקו את המעגל-
.הכול תקין, אדוני-

3
00:01:00,548 --> 00:01:02,250
.אם כך לא ייתכן שהבעיה במסך

4
00:01:04,517 --> 00:01:07,120
אין ספק שיש שם משהו
.שמתקרב אלינו, קפטן

5
00:01:09,489 --> 00:01:11,658
.ייתכן שאלה המטאוראידים האלה

6
00:01:11,992 --> 00:01:14,561
.לא, זה משהו אחר. יש שם משהו בכל זאת

7
00:01:24,938 --> 00:01:27,641
.הוא מתקרב במהיר


In [28]:
# def extract_text_from_srt(srt_content):
#     """
#     Extracts only the text content from an SRT subtitle string.
#     Removes the index numbers, time codes, and empty lines.

#     Args:
#         srt_content (str): The full content of the subtitle file in SRT format.

#     Returns:
#         str: A single long string containing only the subtitle text.
#     """
#     # Step 1: Remove index lines (lines with just numbers)
#     srt_content = re.sub(r'^\d+\s*$', '', srt_content, flags=re.MULTILINE)

#     # Step 2: Remove time codes in the format --> (e.g., 00:00:26,908 --> 00:00:28,910)
#     srt_content = re.sub(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}', '', srt_content)

#     # Step 3: Remove any blank lines caused by the above replacements
#     srt_content = re.sub(r'\n+', '\n', srt_content).strip()

#     # Step 4: Join all lines into a single long string with spaces
#     text_only = ' '.join(line.strip() for line in srt_content.splitlines() if line.strip())

#     return text_only

In [7]:
def clean_srt_to_paragraph(srt_content):
    """
    Removes line numbers, timecodes, and collapses subtitles into a single paragraph.

    Args:
        srt_content (str): Raw subtitle content in SRT format.

    Returns:
        str: Cleaned and consolidated text without line breaks.
    """
    # Step 1: Remove line numbers (lines with only digits)
    srt_content = re.sub(r'^\d+\s*$', '', srt_content, flags=re.MULTILINE)

    # Step 2: Remove timecode lines (contains --> with timestamps)
    srt_content = re.sub(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}', '', srt_content)

    # Step 3: Remove extra blank lines
    srt_content = re.sub(r'\n+', '\n', srt_content)

    # Step 4: Replace all newlines with spaces to consolidate into a paragraph
    srt_content = srt_content.replace('\n', ' ')

    # Step 5: Remove extra spaces (if any)
    cleaned_content = re.sub(r'\s+', ' ', srt_content).strip()

    return cleaned_content

In [8]:
def get_textsub_by_id(sub_id):
    return clean_srt_to_paragraph(get_and_process_subtitle(sub_id))

In [92]:
# Fetch and display subtitle file for sub_id 188032
sub_id = 68938.0
print(get_textsub_by_id(sub_id))

תורגם וסונכרן ע"י שוביקס !למטרת למידה בלבד ...קפטן .מחשב, להפסיק מוזיקה ?לא הערתי אותך .זה בסדר, דוקטור ‏15 דקות של שינה .זה באמת כל מה שאני צריכה .קפה, שחור אולי כדאי שתשים לב יותר לטיסה .ופחות יותר לשירה כהולוגרמה, אני יכול להתמודד .עם מגוון מטלות בבת אחת ,בנוסף להטסת הפלייר אני כותב מאמר על לחץ פיזיולוגי בשהייה ממושכת בטיסת חלל ומצלם תמונות הולוגרפיות .של ערפילית המוטרה ,עם הולוגרמה בסיפון ?מי צריך צוות ,לעולם לא אודה בזה בפני אף אחד אבל היתה תקופה שהייתי נותן הכל ,כדי להיות בשר ודם אבל הגעתי למסקנה שלהיות הולוגרמה .זה נעלה יותר ?באמת .בלי לפגוע .תמיד אהנה מחברת האורגניים למעשה, אני שמח שיכולת לבוא .לכנס הרפואי איתי .זהו עונג לבלות איתך למעשה, קיוויתי שאולי נעשה .דברים כאלה לעתים קרובות יותר .מממ .אנחנו עוברים דרך מערבולת תת חלל זה עשוי להיות שובל .של ספינה נוספת .לא .אין כלום בחיישנים זאת בוודאי רק גזירה כבידתית .מהערפילית .אולי כדאי שאקח את השליטה .שבי ותירגעי, קפטן .יש לך הולוגרמה בהיגוי תורגם וסונכרן ע"י שוביקס !למטרת למידה בלבד מב"כ וויאג'ר - עונה 7 פרק 24 -א י ש ה ר נ ס נ ס- ה

In [65]:
def count_tokens(text):
    """
    Counts the number of tokens (words) in a given text.

    Args:
        text (str): Input text.

    Returns:
        int: The number of tokens.
    """
    tokens = text.split()  # Split by whitespace
    return len(tokens)

In [53]:
print(f"Number of tokens: {count_tokens(get_textsub_by_id(sub_id))}")

Number of tokens: 4109


In [82]:
# def add_subtitles_to_df(df):
#     """
#     Processes each row in the DataFrame, fetches and cleans subtitles 
#     using get_textsub_by_id, and adds them to a new 'subtitles' column.
    
#     Args:
#         df (pd.DataFrame): The input DataFrame containing a 'sub_id' column.
        
#     Returns:
#         pd.DataFrame: The updated DataFrame with the 'subtitles' column added.
#     """
#     subtitles = []  # List to hold the processed subtitles
    
#     # Use tqdm to show progress
#     for sub_id in tqdm(df['sub_id'], desc="Processing subtitles"):
#         if pd.notnull(sub_id):  # Process only if sub_id is not null
#             try:
#                 cleaned_text = get_textsub_by_id(sub_id)
#                 subtitles.append(cleaned_text if cleaned_text else None)
#             except Exception as e:
#                 print(f"Error processing sub_id {sub_id}: {e}")
#                 subtitles.append(None)  # Add None in case of errors
#         else:
#             subtitles.append(None)  # Add None for null sub_ids
    
#     # Add the subtitles column to the DataFrame
#     df['subtitles'] = subtitles
#     return df

In [85]:
def fetch_and_clean_subtitle(sub_id):
    """
    Fetches and cleans subtitles for a given sub_id.
    
    Args:
        sub_id (float): The subtitle ID.
        
    Returns:
        str or None: Cleaned subtitle content or None if failed.
    """
    try:
        if pd.notnull(sub_id):  # Process only if sub_id is not null
            cleaned_text = get_textsub_by_id(sub_id)
            return cleaned_text if cleaned_text else None
    except Exception as e:
        tqdm.write(f"Error processing sub_id {sub_id}: {e}")
    return None

def add_subtitles_to_df(df, max_workers=10):
    """
    Processes each row in the DataFrame concurrently, fetches and cleans subtitles, 
    and adds them to a new 'subtitles' column.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing a 'sub_id' column.
        max_workers (int): The maximum number of threads to use for parallel processing.
        
    Returns:
        pd.DataFrame: The updated DataFrame with the 'subtitles' column added.
    """
    subtitles = [None] * len(df)  # Pre-initialize a list to store subtitles
    
    # Create a ThreadPoolExecutor for parallel requests
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_index = {
            executor.submit(fetch_and_clean_subtitle, row['sub_id']): idx
            for idx, row in df.iterrows() if pd.notnull(row['sub_id'])
        }
        
        # Track progress with tqdm
        for future in tqdm(as_completed(future_to_index), total=len(future_to_index), desc="Processing subtitles"):
            idx = future_to_index[future]
            try:
                subtitles[idx] = future.result()
            except Exception as e:
                tqdm.write(f"Error at index {idx}: {e}")
    
    # Add the subtitles column to the DataFrame
    df['subtitles'] = subtitles
    return df


In [86]:
tqdm.pandas()
df_imdb_sub = add_subtitles_to_df(df_imdb_sub)

Processing subtitles:  42%|████▏     | 27908/67119 [12:36<13:54, 46.98it/s]

Error while fetching subtitle for sub_id 301525.0: HTTPSConnectionPool(host='wizdom.xyz', port=443): Max retries exceeded with url: /api/files/sub/301525.0 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fdb8e086600>: Failed to resolve 'wizdom.xyz' ([Errno -2] Name or service not known)"))
Error processing sub_id 301525.0: expected string or bytes-like object, got 'NoneType'
Error while fetching subtitle for sub_id 301277.0: HTTPSConnectionPool(host='wizdom.xyz', port=443): Max retries exceeded with url: /api/files/sub/301277.0 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fdb8e09c7a0>: Failed to resolve 'wizdom.xyz' ([Errno -2] Name or service not known)"))
Error processing sub_id 301277.0: expected string or bytes-like object, got 'NoneType'
Error while fetching subtitle for sub_id 301769.0: HTTPSConnectionPool(host='wizdom.xyz', port=443): Max retries exceeded with url: /api/files/sub/301769.0 (Caused by NameResolut

Processing subtitles:  53%|█████▎    | 35586/67119 [16:09<15:03, 34.91it/s]

Error processing sub_id 80789.0: expected string or bytes-like object, got 'NoneType'


Processing subtitles:  55%|█████▍    | 36588/67119 [16:37<14:14, 35.75it/s]

...

Processing subtitles:  76%|███████▌  | 50846/67119 [23:19<07:18, 37.11it/s]

Error while fetching subtitle for sub_id 213123.0: HTTPSConnectionPool(host='wizdom.xyz', port=443): Max retries exceeded with url: /api/files/sub/213123.0 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
Error processing sub_id 213123.0: expected string or bytes-like object, got 'NoneType'


Processing subtitles:  92%|█████████▏| 61936/67119 [28:33<02:11, 39.44it/s]

Error while fetching subtitle for sub_id 7940.0: 500 Server Error: Internal Server Error for url: https://wizdom.xyz/api/files/sub/7940.0
Error processing sub_id 7940.0: expected string or bytes-like object, got 'NoneType'


Processing subtitles: 100%|██████████| 67119/67119 [30:59<00:00, 36.10it/s]


In [87]:
df_imdb_sub.head()

,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode,sub_id,subtitles
0,tt0059753,tt0060028,1,0,The Cage,63.0,Star Trek,8.4,95360.0,7.6,7428.0,1966.0,188032.0,"הכלוב .תבדקו את המעגל- .הכול תקין, אדוני- .אם ..."
1,tt0061027,tt0060028,1,3,Where No Man Has Gone Before,50.0,Star Trek,8.4,95360.0,7.7,6688.0,1966.0,181335.0,",יומן הקפטן .זמן כוכבים 1312.4 ,הבלתי אפשרי קר..."
2,tt0066931,tt1466074,1,3,Dead Weight,76.0,Columbo,8.3,43873.0,6.9,3538.0,1971.0,94598.0,.דוטון .גנרל הוליסטר .היכנס ?רוצה לשתות- .לא. ...
3,tt0073677,tt0071054,3,17,The Secret of Bigfoot: Part 1,47.0,The Six Million Dollar Man,7.1,13511.0,7.6,399.0,1976.0,297110.0,***האיש השווה מיליונים*** תורגם סונכרן וקודד ע...
4,tt0084218,tt0083437,1,1,Knight of the Phoenix: Part 1,45.0,Knight Rider,6.9,38628.0,7.3,2272.0,1982.0,5919.0,"<i>- קונסוליה זרה, ארה""ב -</i> ?תסלחי לי לרגע ..."


In [107]:
df_imdb_sub['subtitles'].isnull().sum()

5

In [108]:
df_imdb_sub = df_imdb_sub.dropna(subset=['subtitles'])

In [109]:
df_imdb_sub['subtitles'].isnull().sum()

0

In [110]:
df_imdb_sub.to_csv(os.path.join(prefix_path, "data", "imdb_text_subtitles.csv"), index=False)

### Additional tests

In [136]:
df_imdb[df_imdb['tconst_season'] == 'tt0094574']
# df_imdb[(df_imdb['tconst_season'] == 'tt0096697') & (df_imdb['episodeNumber'] == 6.0) ]
# df_imdb[(df_imdb['tconst_season'] == 'tt6809396') & (df_imdb['seasonNumber'] == 1.0) ]


,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode
245368,tt0737492,tt0094574,1.0,27.0,Episode #1.27,NaN,Unsolved Mysteries,8.3,6880.0,8.4,23.0,1989.0
245369,tt0737493,tt0094574,12.0,8.0,Internet Abduction,44.0,Unsolved Mysteries,8.3,6880.0,7.8,11.0,2001.0
245370,tt0737494,tt0094574,3.0,17.0,Episode #3.17,42.0,Unsolved Mysteries,8.3,6880.0,7.6,22.0,1991.0
245371,tt0737495,tt0094574,11.0,2.0,Episode #11.2,42.0,Unsolved Mysteries,8.3,6880.0,8.0,23.0,1999.0
245372,tt0737496,tt0094574,4.0,15.0,Episode #4.15,42.0,Unsolved Mysteries,8.3,6880.0,8.1,27.0,1991.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7388134,tt6410312,tt0094574,1.0,28.0,Episode #1.28,NaN,Unsolved Mysteries,8.3,6880.0,8.0,19.0,1989.0
7779469,tt7525922,tt0094574,NaN,NaN,1 episode,NaN,Unsolved Mysteries,8.3,6880.0,NaN,NaN,1990.0
7787390,tt7549344,tt0094574,NaN,NaN,Episode dated 20 January 1987,60.0,Unsolved Mysteries,8.3,6880.0,8.0,15.0,1987.0
8678494,tt9914702,tt0094574,NaN,NaN,Episode dated 5 February 1988,NaN,Unsolved Mysteries,8.3,6880.0,7.1,32.0,1988.0


In [15]:
# df_monitor = df_imdb[df_imdb['title_series'] == 'Monitor']
# # title_episode
# df_monitor[df_monitor['title_episode'] == 'Elgar: Portrait of a Composer']

df_imdb[(df_imdb['title_series'] == 'Monitor') & (df_imdb['title_episode'] =='Elgar: Portrait of a Composer')]


,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode
112,tt0055952,tt0259792,6.0,5.0,Elgar: Portrait of a Composer,55.0,Monitor,8.6,103.0,7.8,284.0,1962.0
